<a href="https://colab.research.google.com/github/AdityaBala05/YOLO-image-classification/blob/main/Yolo_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO Model for Skin Disease Classification

In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.6/911.6 kB 11.7 MB/s eta 0:00:00


# Importing Libraries and Creating train-test split

In [ ]:
import os
from sklearn.model_selection import train_test_split
import shutil
import torch
from ultralytics import YOLO
from google.colab import drive
from PIL import Image
import matplotlib.pyplot as plt

drive.mount('/content/drive')

data_path = "/content/drive/My Drive/Preprocessed/Preprocessed"
output_path = "/content/processed_dataset"
os.makedirs(output_path, exist_ok=True)

for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_path, split), exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Split data into train, val, and test (80% train, 10% val, 10% test)

In [ ]:
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

for class_name in os.listdir(data_path):
    class_path = os.path.join(data_path, class_name)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        train_images, test_val_images = train_test_split(images, test_size=(val_ratio + test_ratio), random_state=42)
        val_images, test_images = train_test_split(test_val_images, test_size=(test_ratio / (val_ratio + test_ratio)), random_state=42)

        for split, split_images in zip(['train', 'val', 'test'], [train_images, val_images, test_images]):
            split_class_path = os.path.join(output_path, split, class_name)
            os.makedirs(split_class_path, exist_ok=True)
            for img in split_images:
                img_path = os.path.join(class_path, img)
                try:
                    shutil.copy(img_path, os.path.join(split_class_path, img))
                except Exception as e:
                    print(f"Error processing image {img_path}: {e}")

print("Dataset prepared with train, val, and test splits.")

Dataset prepared with train, val, and test splits.


# Copying dataset with split to "/content/drive/My Drive/YoloDataset"



In [ ]:
import shutil
import os

# Define Google Drive directory
drive_path = "/content/drive/My Drive/YoloDataset"

# Ensure directory exists
os.makedirs(drive_path, exist_ok=True)

# Copy dataset to Google Drive
shutil.copytree(output_path, os.path.join(drive_path, "dataset"), dirs_exist_ok=True)

print("Dataset copied to Google Drive successfully!")


Dataset copied to Google Drive successfully!


# YOLOV8

# Nano

In [ ]:
model = YOLO('yolov8n-cls.pt')

100%|██████████| 5.31M/5.31M [00:00<00:00, 288MB/s]


In [ ]:
results = model.train(
    data=output_path,  # Path to dataset
    epochs=100,         # Number of epochs
    imgsz=300,         # Image size
    batch=32,          # Batch size
    patience=7,        # Early stopping after 5 epochs without improvement
    name='custom_yolov8_classification'  # Run name for logs
)

Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/processed_dataset, epochs=100, time=None, patience=7, batch=32, imgsz=300, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_yolov8_classification, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True

100%|██████████| 5.35M/5.35M [00:00<00:00, 316MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[300] must be multiple of max stride 32, updating to [320]


train: Scanning /content/processed_dataset/train... 3943 images, 0 corrupt: 100%|██████████| 3943/3943 [00:01<00:00, 3830.11it/s]

train: New cache created: /content/processed_dataset/train.cache


...

     41/100     0.872G     0.1312          7        320: 100%|██████████| 124/124 [00:44<00:00,  2.79it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.75it/s]

                   all      0.878          1
EarlyStopping: Training stopped early as no improvement observed in last 7 epochs. Best results observed at epoch 34, best model saved as best.pt.
To update EarlyStopping(patience=7) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



41 epochs completed in 0.556 hours.
Optimizer stripped from runs/classify/custom_yolov8_classification/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/custom_yolov8_classification/weights/best.pt, 3.0MB

Validating runs/classify/custom_yolov8_classification/weights/best.pt...
Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1,440,004 parameters, 0 gradients, 3.3 GFLOPs
train: /content/processed_dataset/train... found 3943 images in 4 classes ✅ 
val: /content/processed_dataset/val... found 492 images in 4 classes ✅ 
test: /content/processed_dataset/test... found 496 images in 4 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.68it/s]


                   all      0.886          1
Speed: 0.2ms preprocess, 0.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/custom_yolov8_classification


# Copying results to "/content/drive/MyDrive/Runs/Yolov8n-100e/"

In [ ]:
!scp -r /content/runs/classify/custom_yolov8_classification "/content/drive/MyDrive/Runs/Yolov8n-100e/"

# YOLO Small

In [ ]:
model = YOLO('yolov8s-cls.pt')  # Using YOLOv8 nano pretrained on ImageNet

100%|██████████| 12.3M/12.3M [00:00<00:00, 47.6MB/s]


In [ ]:
# Step 4: Train the Model with Early Stopping
results = model.train(
    data=output_path,  # Path to dataset
    epochs=100,         # Number of epochs
    imgsz=300,         # Image size
    batch=32,          # Batch size
    patience=7,        # Early stopping after 5 epochs without improvement
    name='custom_yolov8_classification'  # Run name for logs
)

Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/processed_dataset, epochs=100, time=None, patience=7, batch=32, imgsz=300, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_yolov8_classification2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

train: Scanning /content/processed_dataset/train... 3943 images, 0 corrupt: 100%|██████████| 3943/3943 [00:00<?, ?it/s]
val: Scanning /content/processed_dataset/val... 492 images, 0 corrupt: 100%|██████████| 492/492 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 320 train, 320 val
Using 2 dataloader workers
Logging results to runs/classify/custom_yolov8_classification2
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100      1.46G      1.087          7        320: 100%|██████████| 124/124 [00:47<00:00,  2.61it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  2.85it/s]

                   all      0.736          1


...

     18/100      1.49G     0.2113          7        320: 100%|██████████| 124/124 [00:44<00:00,  2.76it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.50it/s]

                   all      0.872          1
EarlyStopping: Training stopped early as no improvement observed in last 7 epochs. Best results observed at epoch 11, best model saved as best.pt.
To update EarlyStopping(patience=7) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



18 epochs completed in 0.243 hours.
Optimizer stripped from runs/classify/custom_yolov8_classification2/weights/last.pt, 10.3MB
Optimizer stripped from runs/classify/custom_yolov8_classification2/weights/best.pt, 10.3MB

Validating runs/classify/custom_yolov8_classification2/weights/best.pt...
Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-cls summary (fused): 73 layers, 5,080,324 parameters, 0 gradients, 12.5 GFLOPs
train: /content/processed_dataset/train... found 3943 images in 4 classes ✅ 
val: /content/processed_dataset/val... found 492 images in 4 classes ✅ 
test: /content/processed_dataset/test... found 496 images in 4 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.48it/s]


                   all      0.878          1
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/custom_yolov8_classification2


In [ ]:
ls "/content/runs/classify/custom_yolov8_classification"

args.yaml                                          train_batch0.jpg       val_batch1_pred.jpg
confusion_matrix_normalized.png                    train_batch1.jpg       val_batch2_labels.jpg
confusion_matrix.png                               train_batch2.jpg       val_batch2_pred.jpg
events.out.tfevents.1736923661.cc17c0e1f52f.482.0  val_batch0_labels.jpg  weights/
results.csv                                        val_batch0_pred.jpg
results.png                                        val_batch1_labels.jpg


In [ ]:
!scp -r /content/runs/classify/custom_yolov8_classification2 "/content/drive/MyDrive/Runs/Yolov8s-100e/"

# YOLO Medium

In [ ]:
model = YOLO('yolov8m-cls.pt')  # Using YOLOv8 nano pretrained on ImageNet

100%|██████████| 32.7M/32.7M [00:00<00:00, 40.1MB/s]


In [ ]:
# Step 4: Train the Model with Early Stopping
results = model.train(
    data=output_path,  # Path to dataset
    epochs=100,         # Number of epochs
    imgsz=300,         # Image size
    batch=32,          # Batch size
    patience=7,        # Early stopping after 5 epochs without improvement
    name='custom_yolov8_classification'  # Run name for logs
)

Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=/content/processed_dataset, epochs=100, time=None, patience=7, batch=32, imgsz=300, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_yolov8_classification3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

train: Scanning /content/processed_dataset/train... 3943 images, 0 corrupt: 100%|██████████| 3943/3943 [00:00<?, ?it/s]
val: Scanning /content/processed_dataset/val... 492 images, 0 corrupt: 100%|██████████| 492/492 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 38 weight(decay=0.0), 39 weight(decay=0.0005), 39 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 320 train, 320 val
Using 2 dataloader workers
Logging results to runs/classify/custom_yolov8_classification3
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100      2.62G      1.068          7        320: 100%|██████████| 124/124 [00:48<00:00,  2.55it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  2.67it/s]

                   all      0.742          1


...

     26/100      2.54G     0.1344          7        320: 100%|██████████| 124/124 [00:46<00:00,  2.67it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.58it/s]

                   all      0.896          1
EarlyStopping: Training stopped early as no improvement observed in last 7 epochs. Best results observed at epoch 19, best model saved as best.pt.
To update EarlyStopping(patience=7) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



26 epochs completed in 0.370 hours.
Optimizer stripped from runs/classify/custom_yolov8_classification3/weights/last.pt, 31.7MB
Optimizer stripped from runs/classify/custom_yolov8_classification3/weights/best.pt, 31.7MB

Validating runs/classify/custom_yolov8_classification3/weights/best.pt...
Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,767,780 parameters, 0 gradients, 41.6 GFLOPs
train: /content/processed_dataset/train... found 3943 images in 4 classes ✅ 
val: /content/processed_dataset/val... found 492 images in 4 classes ✅ 
test: /content/processed_dataset/test... found 496 images in 4 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.29it/s]


                   all      0.898          1
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/custom_yolov8_classification3


In [ ]:
!scp -r /content/runs/classify/custom_yolov8_classification3 "/content/drive/MyDrive/Runs/Yolov8m-100e/"

# YOLO Large

In [ ]:
model = YOLO('yolov8l-cls.pt')  # Using YOLOv8 nano pretrained on ImageNet

100%|██████████| 71.7M/71.7M [00:03<00:00, 22.6MB/s]


In [ ]:
# Step 4: Train the Model with Early Stopping
results = model.train(
    data=output_path,  # Path to dataset
    epochs=100,         # Number of epochs
    imgsz=300,         # Image size
    batch=32,          # Batch size
    patience=7,        # Early stopping after 5 epochs without improvement
    name='custom_yolov8_classification'  # Run name for logs
)

Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=/content/processed_dataset, epochs=100, time=None, patience=7, batch=32, imgsz=300, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_yolov8_classification4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

train: Scanning /content/processed_dataset/train... 3943 images, 0 corrupt: 100%|██████████| 3943/3943 [00:00<?, ?it/s]
val: Scanning /content/processed_dataset/val... 492 images, 0 corrupt: 100%|██████████| 492/492 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 50 weight(decay=0.0), 51 weight(decay=0.0005), 51 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 320 train, 320 val
Using 2 dataloader workers
Logging results to runs/classify/custom_yolov8_classification4
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100      4.41G      1.061          7        320: 100%|██████████| 124/124 [00:54<00:00,  2.28it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.15it/s]

                   all      0.754          1


...

     33/100       4.2G    0.09463          7        320: 100%|██████████| 124/124 [00:49<00:00,  2.48it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.78it/s]

                   all      0.882          1
EarlyStopping: Training stopped early as no improvement observed in last 7 epochs. Best results observed at epoch 26, best model saved as best.pt.
To update EarlyStopping(patience=7) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



33 epochs completed in 0.573 hours.
Optimizer stripped from runs/classify/custom_yolov8_classification4/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/custom_yolov8_classification4/weights/best.pt, 72.6MB

Validating runs/classify/custom_yolov8_classification4/weights/best.pt...
Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-cls summary (fused): 133 layers, 36,189,700 parameters, 0 gradients, 98.7 GFLOPs
train: /content/processed_dataset/train... found 3943 images in 4 classes ✅ 
val: /content/processed_dataset/val... found 492 images in 4 classes ✅ 
test: /content/processed_dataset/test... found 496 images in 4 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.12it/s]


                   all      0.894          1
Speed: 0.1ms preprocess, 2.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/custom_yolov8_classification4


In [ ]:
!scp -r /content/runs/classify/custom_yolov8_classification4 "/content/drive/MyDrive/Runs/Yolov8l-100e/"

# YOLO XLarge

In [ ]:
model = YOLO('yolov8x-cls.pt')  # Using YOLOv8 xl pretrained on ImageNet

100%|██████████| 110M/110M [00:06<00:00, 18.0MB/s]


In [ ]:
output_path = "/content/drive/My Drive/YoloDataset/dataset"

In [ ]:
results = model.train(
    data=output_path,  # Path to dataset
    epochs=100,         # Number of epochs
    imgsz=300,         # Image size
    batch=32,          # Batch size
    patience=7,        # Early stopping after 5 epochs without improvement
    name='custom_yolov8_classification'  # Run name for logs
)

Ultralytics 8.3.62 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8x-cls.pt, data=/content/drive/My Drive/YoloDataset/dataset, epochs=100, time=None, patience=7, batch=32, imgsz=300, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_yolov8_classification3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False

100%|██████████| 5.35M/5.35M [00:00<00:00, 101MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[300] must be multiple of max stride 32, updating to [320]


train: Scanning /content/drive/My Drive/YoloDataset/dataset/train... 3943 images, 0 corrupt: 100%|██████████| 3943/3943 [00:25<00:00, 156.13it/s]


train: New cache created: /content/drive/My Drive/YoloDataset/dataset/train.cache


...

     39/100      5.59G    0.07758          7        320: 100%|██████████| 124/124 [01:08<00:00,  1.80it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:05<00:00,  1.39it/s]

                   all      0.896          1


EarlyStopping: Training stopped early as no improvement observed in last 7 epochs. Best results observed at epoch 32, best model saved as best.pt.
To update EarlyStopping(patience=7) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

39 epochs completed in 0.869 hours.
Optimizer stripped from runs/classify/custom_yolov8_classification3/weights/last.pt, 112.5MB
Optimizer stripped from runs/classify/custom_yolov8_classification3/weights/best.pt, 112.5MB

Validating runs/classify/custom_yolov8_classification3/weights/best.pt...
Ultralytics 8.3.62 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-cls summary (fused): 133 layers, 56,128,324 parameters, 0 gradients, 153.8 GFLOPs
train: /content/drive/My Drive/YoloDataset/dataset/train... found 3943 images in 4 classes ✅ 
val: /content/drive/My Drive/YoloDataset/dataset/val... found 492 images in 4 classes ✅ 
test: /content/drive/My Drive/YoloDataset/dataset/test... found 496 ima

               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:03<00:00,  2.24it/s]


                   all      0.907          1
Speed: 0.1ms preprocess, 4.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/custom_yolov8_classification3


In [ ]:
!scp -r /content/runs/classify/custom_yolov8_classification3 "/content/drive/MyDrive/YoloDataset"

# YOLO11

# Nano

In [ ]:
model = YOLO('yolo11n-cls.pt')  # Using YOLOv8 xl pretrained on ImageNet

100%|██████████| 5.52M/5.52M [00:00<00:00, 67.1MB/s]


In [ ]:
results = model.train(
    data=output_path,  # Path to dataset
    epochs=100,         # Number of epochs
    imgsz=300,         # Image size
    batch=32,          # Batch size
    patience=7,        # Early stopping after 5 epochs without improvement
    name='custom_yolov8_classification'  # Run name for logs
)

Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/content/processed_dataset, epochs=100, time=None, patience=7, batch=32, imgsz=300, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_yolov8_classification2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

train: Scanning /content/processed_dataset/train... 3943 images, 0 corrupt: 100%|██████████| 3943/3943 [00:00<?, ?it/s]
val: Scanning /content/processed_dataset/val... 492 images, 0 corrupt: 100%|██████████| 492/492 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 320 train, 320 val
Using 2 dataloader workers
Logging results to runs/classify/custom_yolov8_classification2
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100      1.78G      1.034          7        320: 100%|██████████| 124/124 [00:45<00:00,  2.72it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  2.98it/s]

                   all      0.762          1


...

     18/100     0.883G     0.3181          7        320: 100%|██████████| 124/124 [00:41<00:00,  3.00it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.89it/s]

                   all      0.841          1
EarlyStopping: Training stopped early as no improvement observed in last 7 epochs. Best results observed at epoch 11, best model saved as best.pt.
To update EarlyStopping(patience=7) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



18 epochs completed in 0.234 hours.
Optimizer stripped from runs/classify/custom_yolov8_classification2/weights/last.pt, 3.2MB
Optimizer stripped from runs/classify/custom_yolov8_classification2/weights/best.pt, 3.2MB

Validating runs/classify/custom_yolov8_classification2/weights/best.pt...
Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n-cls summary (fused): 112 layers, 1,531,148 parameters, 0 gradients, 3.2 GFLOPs
train: /content/processed_dataset/train... found 3943 images in 4 classes ✅ 
val: /content/processed_dataset/val... found 492 images in 4 classes ✅ 
test: /content/processed_dataset/test... found 496 images in 4 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:01<00:00,  4.34it/s]


                   all      0.868          1
Speed: 0.3ms preprocess, 0.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/custom_yolov8_classification2


In [ ]:
!scp -r /content/runs/classify/custom_yolov8_classification2 "/content/drive/MyDrive/Runs/Yolo11n-100e"

# Completion of Training

# Model Evaluation

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.4/911.4 kB 48.8 MB/s eta 0:00:00


In [ ]:
"/content/drive/MyDrive/Runs/Yolo11n-100e"
"/content/drive/MyDrive/Runs/Yolov8l-100e/"
"/content/drive/MyDrive/Runs/Yolov8m-100e/"
"/content/drive/MyDrive/Runs/Yolov8s-100e/"
"/content/drive/MyDrive/Runs/Yolov8n-100e/"

# Evaluation of nano model

In [ ]:
from ultralytics import YOLO

# Load trained model
model = YOLO("/content/drive/MyDrive/Runs/Yolov8x-100e/custom_yolov8_classification/weights/best.pt")

# Evaluate on validation set
metrics = model.val()

# Print accuracy
print(f"Overall Accuracy of Val data: {metrics.top1:.2f}%")

# Validate on the test set
metrics = model.val(split="test")

# Print overall accuracy
print(f"Overall Accuracy of Test data: {metrics.top1:.4f}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Runs/Yolov8x-100e/custom_yolov8_classification/weights/best.pt'

In [ ]:
!scp -r /content/runs/classify/val "/content/drive/MyDrive/Validation_YOLOv8x"

In [ ]:
ls "/content/drive/MyDrive/YoloDataset/custom_yolov8_classification3"

args.yaml                                           train_batch0.jpg       val_batch1_pred.jpg
confusion_matrix_normalized.png                     train_batch1.jpg       val_batch2_labels.jpg
confusion_matrix.png                                train_batch2.jpg       val_batch2_pred.jpg
events.out.tfevents.1737100361.b8c611514bbe.1005.0  val_batch0_labels.jpg  weights/
results.csv                                         val_batch0_pred.jpg
results.png                                         val_batch1_labels.jpg


#best.pt for XLarge YOLOV8 model
"/content/drive/MyDrive/YoloDataset/custom_yolov8_classification3/weights/best.pt"

In [ ]:
from ultralytics import YOLO

# Load the trained classification model
model = YOLO("/content/drive/MyDrive/YoloDataset/custom_yolov8_classification3/weights/best.pt")  # Replace with your model path

# Evaluate on validation set
metrics = model.val()

# Print accuracy
print(f"Overall Accuracy of Val data: {metrics.top1:.2f}%")

# Validate on the test set
metrics = model.val(split="test")

# Print overall accuracy
print(f"Overall Accuracy of Test data: {metrics.top1:.4f}")


Ultralytics 8.3.65 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-cls summary (fused): 133 layers, 56,128,324 parameters, 0 gradients, 153.8 GFLOPs
train: /content/drive/My Drive/YoloDataset/dataset/train... found 3943 images in 4 classes ✅ 
val: /content/drive/My Drive/YoloDataset/dataset/val... found 492 images in 4 classes ✅ 
test: /content/drive/My Drive/YoloDataset/dataset/test... found 496 images in 4 classes ✅ 


test: Scanning /content/drive/My Drive/YoloDataset/dataset/test... 496 images, 0 corrupt: 100%|██████████| 496/496 [05:06<00:00,  1.62it/s]

test: New cache created: /content/drive/My Drive/YoloDataset/dataset/test.cache



               classes   top1_acc   top5_acc:   6%|▋         | 2/31 [00:01<00:16,  1.78it/s]

               classes   top1_acc   top5_acc:  23%|██▎       | 7/31 [00:02<00:05,  4.28it/s]
100%|██████████| 755k/755k [00:00<00:00, 74.2MB/s]
               classes   top1_acc   top5_acc: 100%|██████████| 31/31 [00:06<00:00,  4.84it/s]


                   all      0.887          1
Speed: 0.7ms preprocess, 9.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val
Overall Accuracy: 0.8871
